## IMPORTING LIBRARIES

In [7]:
!pip install geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
    

## GET DATA FROM WEBPAGE

In [9]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})
mum_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [10]:
mum_df.shape

(40, 1)

## GEOGRAPHICAL COORDINATES

In [13]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00534722389655, 72.85580272012932],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21198153260436, 72.83757275783374],
 [19.048530000000028, 72.93222000000003

In [14]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

print(mum_df.shape)
mum_df

(40, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [15]:
mum_df.to_csv("mum_df.csv", index=False)

## CREATING A MAP

In [16]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [26]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='#199e19', fill=True, fill_color='#199e19', fill_opacity=0.8).add_to(map_mum)  
    
map_mum

In [27]:
map_mum.save('map_mum.html')

## USING FOURSQUARE

In [28]:
CLIENT_ID = 'D0D2L4UQSYF5EP34OBOJFTGNM4ITZ5HPBA454DDIJAND0DWR' 
CLIENT_SECRET = '5LQDCX250H3L5QFYLJQGXJK0DSJGSUNVBHPR5O53WIOVZKA3'
VERSION = '20180605'

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: D0D2L4UQSYF5EP34OBOJFTGNM4ITZ5HPBA454DDIJAND0DWR
CLIENT_SECRET:5LQDCX250H3L5QFYLJQGXJK0DSJGSUNVBHPR5O53WIOVZKA3


In [29]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):

    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, radius, LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']

    for venue in results:
        venues.append((neighborhood, lat, long, venue['venue']['name'], venue['venue']['location']['lat'], 
        venue['venue']['location']['lng'], venue['venue']['categories'][0]['name']))

In [30]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2592, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Narayan Sandwich,19.121398,72.850270,Sandwich Place
4,Andheri,19.118459,72.841763,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop


In [31]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,16,16,16,16,16,16
Baiganwadi,10,10,10,10,10,10
Bandra,100,100,100,100,100,100
Bhandup,21,21,21,21,21,21
Borivali,100,100,100,100,100,100
Charkop,49,49,49,49,49,49
Chembur,97,97,97,97,97,97
Dahisar,61,61,61,61,61,61


In [32]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 197 uniques categories.


In [34]:
venues_df['VenueCategory'].unique()

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop', 'Sandwich Place',
       'Coffee Shop', 'Falafel Restaurant', 'Pub', 'Juice Bar',
       'Pizza Place', 'Multiplex', 'Seafood Restaurant',
       'Fast Food Restaurant', 'Breakfast Spot', 'Snack Place',
       'Maharashtrian Restaurant', 'Chinese Restaurant', 'Café',
       'American Restaurant', 'Cocktail Bar', 'Bar', 'Diner',
       'Department Store', 'BBQ Joint', "Women's Store",
       'Gym / Fitness Center', 'Spa', 'Electronics Store', 'Lounge',
       'Asian Restaurant', 'Residential Building (Apartment / Condo)',
       'Park', 'Smoke Shop', 'Athletics & Sports', 'Food Truck',
       'Vegetarian / Vegan Restaurant', 'Liquor Store',
       'Martial Arts Dojo', 'Tea Room', 'Fish Market', 'Food', 'Plaza',
       'Mughlai Restaurant', 'Platform', 'Paper / Office Supplies Store',
       'Food & Drink Shop', 'Hot Dog Joint', 'Fried Chicken Joint', 'Gym',
       'Dessert Shop', 'Sports Club', 'Gourmet Shop', 'Korean Restaurant',


In [35]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## ANALYZING NEIGHBOURHOOD

In [36]:
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(2592, 198)


,Neighborhoods,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Wine Shop,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(39, 198)


,Neighborhoods,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Wine Shop,Women's Store
0,Andheri,0.0000,0.010000,0.00,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.010000,0.00,0.020000,0.0000,0.050000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.040000,0.00,0.00,0.00,0.060000,0.000000,0.01,0.020000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.020000,0.000000,0.0000,0.01,0.00,0.000000,0.020000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.040000,0.170000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.020000,0.00,0.000000,0.01,0.020000,0.010000,0.000000,0.01,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00000

In [38]:
len(mum_grouped[mum_grouped["Shopping Mall"] > 0])

15

In [40]:
mum_mall = mum_grouped[["Neighborhoods","Shopping Mall"]]
mum_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.047619


## CLUSTERING NEIGHBOURHOODS

In [41]:
kclusters = 3
mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 2, 0, 2, 0, 0])

In [43]:
mum_merged = mum_mall.copy()
mum_merged["Cluster Labels"] = kmeans.labels_

mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,0
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.000000,0
3,Bandra,0.000000,0
4,Bhandup,0.047619,1


In [44]:
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head()

(39, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,0,19.042830,72.927340
2,Baiganwadi,0.000000,0,19.062940,72.926630
3,Bandra,0.000000,0,19.054370,72.840170
4,Bhandup,0.047619,1,19.145560,72.948560


In [45]:
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(39, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
23,Mankhurd,0.000000,0,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.000000,0,19.211982,72.837573
21,Kurla,0.000000,0,19.064980,72.880690
20,Kausa,0.000000,0,19.127580,72.825390
27,Mumbra,0.000000,0,19.190540,73.022660
18,Kandivali,0.000000,0,19.211982,72.837573
17,Kalyan,0.000000,0,18.953937,72.820367
16,Juhu,0.000000,0,19.014920,72.845220
15,Jogeshwari,0.000000,0,19.137920,72.849410


In [47]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
map_clusters.save('map_clusters.html')

## EXAMINING CLUSTERS

In [49]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
23,Mankhurd,0.0,0,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.0,0,19.211982,72.837573
21,Kurla,0.0,0,19.064980,72.880690
20,Kausa,0.0,0,19.127580,72.825390
27,Mumbra,0.0,0,19.190540,73.022660
18,Kandivali,0.0,0,19.211982,72.837573
17,Kalyan,0.0,0,18.953937,72.820367
16,Juhu,0.0,0,19.014920,72.845220
15,Jogeshwari,0.0,0,19.137920,72.849410


In [50]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
34,Vashi,0.040000,1,19.08465,72.90481
35,Vikhroli,0.030000,1,19.11109,72.92781
19,Kanjurmarg,0.038462,1,19.13138,72.93568
4,Bhandup,0.047619,1,19.14556,72.94856
26,Mulund,0.038462,1,19.17183,72.95565


In [51]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
37,Western Suburbs (Mumbai),0.013158,2,19.197010,72.827680
28,Pestom sagar,0.012048,2,19.070640,72.902170
29,Seven Bungalows,0.010000,2,19.131342,72.816342
13,Goregaon,0.020000,2,19.164550,72.849460
12,Ghatkopar,0.012500,2,19.086523,72.909008
32,Thakur village,0.015385,2,19.210200,72.875410
7,Chembur,0.010309,2,19.062180,72.902410
5,Borivali,0.010000,2,19.229360,72.857510
36,Wadala,0.021505,2,19.017200,72.858170
38,Worli,0.020000,2,19.007440,72.816880


## OBSERVATIONS

Cluster 0 (in red) has no shopping malls. 
Cluster 1 (in purple) has the most number of shopping malls.
Cluster 2 (in cyan) has moderate number of shopping malls.

Therefore, it is advised to open new shopping malls in neighborhoods belonging to cluster 0, where there is no competition.
This will also provide the people living in these neighbourhoods with a means of livelihood.

Property developers can consider opening a shopping mall in neighborhoods belonging to cluster 2 if they believe they can 
stand out from the other malls in the locality, and can fight the moderate competition.

Property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls 
and are suffering from intense competition.